In [1]:
import glob
import pandas as pd
import numpy as np
import os, sys
import collections
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier 

In [3]:
from numba.typed import List
from numba import jit, njit, vectorize

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [5]:
sample_rate = 50  # number of observation per second based on dataset documentation(150 samples in 3 second)

sliding_size = int((1/3) * sample_rate)  # number of skipped datapoints to start next window
print(sliding_size)

16


# Feature sets

In [6]:
@njit()
def mean_crossing_rate(col):
    # col = np.array(values)
    normalized = col - col.mean()  # to make elements of array possitive or negetive
    return ((normalized[:-1] * col[1:]) < 0).sum()  # Zero-Crossing_rate

@njit()
def iqr(window):  # inter-quartile range
    Q1 = np.median(window[:len(window)//2])  # First quartile (Q1) 
    Q3 = np.median(window[len(window)//2:])  # Third quartile (Q3) 
    IQR = Q3 - Q1 # Interquartile range (IQR) 
    return(IQR) 
@njit()
def calc_sma_for_window(data):
    return np.sum(data) / len(data)  
@njit()
def get_min(x):
    m = np.min(x)
    return m
@njit()
def get_max(x):
    m = np.max(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_var(x):
    m = np.var(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_sum(x):
    m = x.sum()
    return m 
@njit()
def get_median(x):
    m = np.median(x)
    return m 
@njit()
def get_std(x):
    m = np.median(x)
    return m 
@njit()
def get_rng(x):
    n = np.max(x)
    m = np.min(x)
    z = n-m
    return z 

def get_rms(x, axis=None):
    return np.sqrt(np.mean(x ** 2, axis=axis))

def calc_sma_for_window(data):
    return np.sum(data) / len(data)


def calc_sma_adv_for_window(data):
    return np.sum(data - np.mean(data) / len(data))


def calc_absolutes_for_list(list):
    return ([abs(i) for i in list])

def get_sma(data): 
    sma_sim = calc_sma_for_window(data)
    sma_adv = calc_sma_adv_for_window(data)

    sma_sim_abs = calc_sma_for_window(calc_absolutes_for_list(data))
    sma_adv_abs = calc_sma_adv_for_window(calc_absolutes_for_list(data))

    return sma_sim, sma_adv, sma_sim_abs, sma_adv_abs

def get_entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

In [7]:
def Energy(frame):
    return sum( [ abs(x)**2 for x in frame ] ) / len(frame)

# Histogram_equalization

In [8]:
def histogram_equalize(im):#https://www.researchgate.net/publication/281118372_NumPy_SciPy_Recipes_for_Image_Processing_Intensity_Normalization_and_Histogram_Equalization
    data = im.copy().flatten()
    hist, bins = np.histogram(data, 256, density=True)
    cdf = hist.cumsum()#normalized cumulative histogram H using NumPy’s inbuilt function cumsum
    cdf = 255*cdf/cdf[-1]  #intensity transformation
    img_eq = np.interp(data, bins[:-1], cdf)
    return img_eq.reshape(im.shape)

# Feature extraction and HISTOGRAM EQUALIZATION on signal

In [9]:
def Features(window):# mean, std,max,min and zero-crossing-rate
    wind = window.iloc[:, :-1]
    winn = np.array(wind)
    win = histogram_equalize(winn) # histogram normalization on signal
    
    features = []
    
    features.append(get_mean(win))
    features.append(get_median(win))
    features.append(get_std(win))
    features.append(get_min(win))
    features.append(get_max(win))
    features.append(get_sum(win))
    features.append(get_entropy(win))
    mean_crossing = [mean_crossing_rate(win[:, i]) for i in range(win.shape[1])]
    features.append(np.array(mean_crossing))
    IQR = iqr(win)
    features.append(np.array(IQR))
    energy_measure = Energy(win)
    features.append(np.array(energy_measure))
 

    
    features = np.hstack(features).tolist()
    
    label = window.iloc[:, -1].mode()[0]  ## select the most frequent label as the label of the window
    features.append(label)
    return features

In [10]:
def windowing_dataset(dataset, win_size, feature_extraction_function, subject_id, overlap=False):
    windowed_dataset = []
    win_count = 0
    if overlap:
        step_size = sliding_size  # for Overlapping technique
    else:
        step_size = win_size  # for Non-overlapping technique

    for index in range(0, dataset.shape[0], step_size):
        start = index
        end = start + win_size
        # to assure all of windows are equal in size
        if (end <= dataset.shape[0]):
            window = dataset.iloc[start:end, :].reset_index(drop=True)
            win_count = win_count + 1
            features = feature_extraction_function(window)

            windowed_dataset.append(features)

    final = pd.DataFrame(windowed_dataset)
    final.insert(0, 'group', subject_id)  # to use in Subject CV
    return final

In [11]:
def Preprocessing(dataset_path, overlapping):
    feature_function = Features
    win_size = 3

    print("Start for win size {}".format(win_size))
    datapoints_per_window = int(win_size * sample_rate)

    print(feature_function.__name__)

    ALL = []
   
    for subject in range(1,18):
        file_path = dataset_path + '\subject{0}_ideal.csv'.format(subject)
        all_cols = []
        
        
#########################################################################################################################
        for i in range(2, 117, 13):# indices of accelarations
            indices = list(range(i, i + 13))
            all_cols.extend(indices)

        all_cols.append(119)  # label index

        tmp_db = pd.read_csv(file_path, header=None, usecols=all_cols, sep='\t')
        tmp_db.columns = list(range(tmp_db.shape[1]))  # re-index the columns

        transformed_db = windowing_dataset(tmp_db, datapoints_per_window, feature_function, subject,
                                                   overlap=overlapping)

        ALL.append(transformed_db)
#########################################################################################################################
       
    #final_dataset = pd.DataFrame()
    ALL_dataset = pd.DataFrame()
    ALL_dataset = ALL_dataset.append(ALL, ignore_index=True)
    
    return ALL_dataset
   

In [12]:
def subject_cross_validation(X, Y, groups, classifier):
    f1 = []
    logo = LeaveOneGroupOut()
    i = 0
    for train_index, test_index in logo.split(X, Y, groups=groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        i += 1
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        f = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
        
        print('Model-',i ,' -',' f1 score: ', f)
       
        f1.append(f)
    return np.mean(f1)

In [13]:
def classifier(dataset, model):
    results = dict()
    win_size = float(3)
    print('window_size = ', win_size,' sec')

    dataset = dataset
    groups = dataset['group']
    X = dataset.iloc[:, 1:-1]
    X = np.array(X)
            
    Y = dataset.iloc[:, -1]
    Y = np.array(Y)

    for model_name, mod in model.items():
        f1 = 0

        f1 = subject_cross_validation(X, Y, groups, mod)

        if win_size in results:
            results[win_size].append(f1)
        else:
            results[win_size] = [f1]


        results = collections.OrderedDict(sorted(results.items()))

        final = []
        col = list(model.keys())
        col.insert(0, "window-size")
        final.append(col)
        for k, v in results.items():
            tmp = []
            tmp.append([k])
            tmp.append(v)
            flattened = [val for sublist in tmp for val in sublist]
            final.append(flattened)

    accuracy = final[1][1]
    
    return accuracy

In [14]:
mod = {'RF': RandomForestClassifier(n_estimators=40, random_state=42, n_jobs=-1)}

In [15]:
dataset_csv_path = r"D:\projec\proj\data"

overlapping = 1  # input 0 for non overlapping, 1 for overlapping

ALL = Preprocessing(dataset_path=dataset_csv_path, overlapping=bool(int(overlapping)))

Start for win size 3
Features


In [16]:
ALL.head(10)

,group,0,1,2,3,4,5,6,7,8,...,233,234,235,236,237,238,239,240,241,242
0,1,138.986865,142.044539,142.044539,0.02906,255.0,2.439219e+06,7.228819,66.0,73.0,...,34566.342657,11748.527744,48352.754281,29976.088217,17579.600644,29721.236361,48618.817661,34064.129640,2675.343913,1
1,1,138.968765,141.888106,141.888106,0.02906,255.0,2.438902e+06,7.228819,65.0,80.0,...,36474.110547,11320.821506,48531.254670,30854.704326,14604.228249,29743.571395,48210.258073,33458.222061,2449.755819,1
2,1,138.788170,141.989666,141.989666,0.02906,255.0,2.435732e+06,7.228819,64.0,86.0,...,41880.151897,12570.290292,48960.628467,30159.140452,15274.459400,28671.385969,48585.826587,33737.549609,2568.721391,1
3,1,137.476308,139.651849,139.651849,0.01453,255.0,2.412709e+06,7.228819,61.0,92.0,...,40044.564126,13674.737045,49057.219249,28121.927494,18368.028455,27522.728445,49009.455859,34138.995375,2850.201634,1
4,1,138.040620,141.544157,141.544157,0.01453,255.0,2.422613e+06,7.228819,62.0,90.0,...,35412.849712,13790.209398,49553.956011,27118.875372,19942.773526,27811.929872,49462.936778,34327.068836,3079.704568,1
5,1,139.248998,144.179159,144.179159,0.01453,255.0,2.443820e+06,7.228819,66.0,80.0,...,35915.186049,12963.253749,49081.774616,27810.470769,19497.749063,28658.600603,49019.526134,34010.756889,3247.190700,1
6,1,139.119312,144.800536,144.800536,0.01453,255.0,2.441544e+06,7.228819,80.0,19.0,...,33676.867144,12835.202443,48808.599673,27700.467307,18517.025699,28559.378053,48678.359365,33313.727983,3253.155828,1
7,1,141.173851,148.797880,148.797880,0.01453,255.0,2.477601e+06,7.228819,91.0,14.0,...,35544.695723,13228.875408,50090.166724,29113.689782,17414.368408,29675.997836,49325.605436,33782.203472,3380.797496,1
8,1,141.185430,149.691917,149.691917,0.01453,255.0,2.477804e+06,7.228819,102.0,18.0,...,40185.048462,14659.495711,50147.748908,28157.900457,19266.878338,28358.706046,49666.906526,33699.094695,3769.112240,1
9,1,141.037305,149.017001,149.017001,0.01453,255.0,2.475205e+06,7.228819,103.0,17.0,...,35568.610931,14548.982646,50825.817934,26784.210935,20559.134059,28014.579140,50199.067258,33717.712476,4038.608257,1


In [17]:
print('ALL_accuracy = ', )
ALL_accuracy = classifier(dataset=ALL, model=mod)
ALL_accuracy 

ALL_accuracy = 
window_size =  3.0  sec
Model- 1  -  f1 score:  0.8916083916083916
Model- 2  -  f1 score:  0.9802608389143461
Model- 3  -  f1 score:  0.858
Model- 4  -  f1 score:  0.9499770536943553
Model- 5  -  f1 score:  0.8951724137931034
Model- 6  -  f1 score:  0.94
Model- 7  -  f1 score:  1.0
Model- 8  -  f1 score:  0.9197388632872504
Model- 9  -  f1 score:  0.9336752136752137
Model- 10  -  f1 score:  0.9375213383407306
Model- 11  -  f1 score:  0.969375
Model- 12  -  f1 score:  0.9745603015075377
Model- 13  -  f1 score:  0.9452441433813152
Model- 14  -  f1 score:  0.9190425906370996
Model- 15  -  f1 score:  0.870073624945864
Model- 16  -  f1 score:  0.8261504747991234
Model- 17  -  f1 score:  0.8836059338151389


0.9231768342587923

In [18]:
d ={'ALL': [ALL_accuracy]}

In [19]:
df_accuracy = pd.DataFrame(data=d)

In [20]:
df_accuracy

,ALL
0,0.923177
